In [1]:
from xgcm import Grid
import pop_tools
import gcsfs
import fsspec as fs
import numpy as np
import xesmf as xe
import xarray as xr
import random
import matplotlib.pyplot as plt
import warnings
from xgcm import Grid
import importlib
import preprocessing
import gcm_filters

warnings.filterwarnings("ignore")

importlib.reload(preprocessing)
from preprocessing import preprocess_data
from gcm_filtering import filter_inputs_dataset
from gcm_filtering import filter_inputs

In [2]:
ds_merged = preprocess_data()
ds_merged

Load Data
Interpolating ocean velocities
Modify units
Load Data
Align in time
Interpolating ocean velocities
Regrid Atmospheric Data
Modify units
Mask nans
Calculate relative wind


<xarray.Dataset> Size: 3TB
Dimensions:       (time: 7305, yt_ocean: 2700, xt_ocean: 3600)
Coordinates:
  * time          (time) object 58kB 0181-01-01 12:00:00 ... 0200-12-31 12:00:00
  * xt_ocean      (xt_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.85 79.95
  * yt_ocean      (yt_ocean) float64 22kB -81.11 -81.07 -81.02 ... 89.94 89.98
    area_t        (yt_ocean, xt_ocean) float64 78MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    dxt           (yt_ocean, xt_ocean) float64 78MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    dyt           (yt_ocean, xt_ocean) float64 78MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    geolat_t      (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    geolon_t      (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    ht            (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    kmt           (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    wet           (yt_ocean, xt_ocean) float64 78MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
Data variables:
    slp           (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    t_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    q_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    wind          (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    surface_temp  (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ocean       (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ocean       (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_relative    (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_relative    (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [3]:
ds_in = ds_merged
ds_in = ds_in.drop(['area_t', 'dxt', 'dyt', 'geolat_t', 'geolon_t', 'ht', 'kmt', 'wet'])
ds_in

<xarray.Dataset> Size: 3TB
Dimensions:       (time: 7305, yt_ocean: 2700, xt_ocean: 3600)
Coordinates:
  * time          (time) object 58kB 0181-01-01 12:00:00 ... 0200-12-31 12:00:00
  * xt_ocean      (xt_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.85 79.95
  * yt_ocean      (yt_ocean) float64 22kB -81.11 -81.07 -81.02 ... 89.94 89.98
Data variables:
    slp           (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    t_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    q_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    wind          (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    surface_temp  (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ocean       (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ocean       (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_relative    (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_relative    (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [4]:
print("Filtering model")
ds_out = filter_inputs_dataset(ds_in, ['yt_ocean', 'xt_ocean'], filter_scale=50, filter_type='gaussian')
ds_out

Filtering model


<xarray.Dataset> Size: 3TB
Dimensions:       (time: 7305, xt_ocean: 3600, yt_ocean: 2700)
Coordinates:
  * time          (time) object 58kB 0181-01-01 12:00:00 ... 0200-12-31 12:00:00
  * xt_ocean      (xt_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.85 79.95
  * yt_ocean      (yt_ocean) float64 22kB -81.11 -81.07 -81.02 ... 89.94 89.98
Data variables:
    slp           (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    t_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    q_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    wind          (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    surface_temp  (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ocean       (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ocean       (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_relative    (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_relative    (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
Attributes:
    filter_scale:  50
    filter_type:   gaussian

In [5]:
ds_lowRes = ds_out.isel(xt_ocean=slice(1500,1600), yt_ocean=slice(1500,1600))
ds_lowRes

<xarray.Dataset> Size: 3GB
Dimensions:       (time: 7305, xt_ocean: 100, yt_ocean: 100)
Coordinates:
  * time          (time) object 58kB 0181-01-01 12:00:00 ... 0200-12-31 12:00:00
  * xt_ocean      (xt_ocean) float64 800B -130.0 -129.9 -129.8 ... -120.2 -120.1
  * yt_ocean      (yt_ocean) float64 800B 24.74 24.83 24.92 ... 33.22 33.3 33.38
Data variables:
    slp           (time, yt_ocean, xt_ocean) float32 292MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    v_ref         (time, yt_ocean, xt_ocean) float32 292MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    u_ref         (time, yt_ocean, xt_ocean) float32 292MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    t_ref         (time, yt_ocean, xt_ocean) float32 292MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    q_ref         (time, yt_ocean, xt_ocean) float32 292MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    wind          (time, yt_ocean, xt_ocean) float32 292MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    surface_temp  (time, yt_ocean, xt_ocean) float32 292MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    u_ocean       (time, yt_ocean, xt_ocean) float32 292MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    v_ocean       (time, yt_ocean, xt_ocean) float32 292MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    u_relative    (time, yt_ocean, xt_ocean) float32 292MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    v_relative    (time, yt_ocean, xt_ocean) float32 292MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
Attributes:
    filter_scale:  50
    filter_type:   gaussian

In [6]:
ds_lowRes_oneYr = ds_lowRes.isel(time=slice(0,365))
ds_lowRes_oneYr

<xarray.Dataset> Size: 161MB
Dimensions:       (time: 365, xt_ocean: 100, yt_ocean: 100)
Coordinates:
  * time          (time) object 3kB 0181-01-01 12:00:00 ... 0181-12-31 12:00:00
  * xt_ocean      (xt_ocean) float64 800B -130.0 -129.9 -129.8 ... -120.2 -120.1
  * yt_ocean      (yt_ocean) float64 800B 24.74 24.83 24.92 ... 33.22 33.3 33.38
Data variables:
    slp           (time, yt_ocean, xt_ocean) float32 15MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    v_ref         (time, yt_ocean, xt_ocean) float32 15MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    u_ref         (time, yt_ocean, xt_ocean) float32 15MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    t_ref         (time, yt_ocean, xt_ocean) float32 15MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    q_ref         (time, yt_ocean, xt_ocean) float32 15MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    wind          (time, yt_ocean, xt_ocean) float32 15MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    surface_temp  (time, yt_ocean, xt_ocean) float32 15MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    u_ocean       (time, yt_ocean, xt_ocean) float32 15MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    v_ocean       (time, yt_ocean, xt_ocean) float32 15MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    u_relative    (time, yt_ocean, xt_ocean) float32 15MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
    v_relative    (time, yt_ocean, xt_ocean) float32 15MB dask.array<chunksize=(3, 100, 100), meta=np.ndarray>
Attributes:
    filter_scale:  50
    filter_type:   gaussian

In [ ]:
# kernel always dies in middle
ds_lowRes_oneYr.to_zarr("gs://leap-persistent/panalluri/filtered_sample_oneYr.zarr")